In [ ]:
from pathlib import Path
import pandas as pd

# Bloque 1 — Rutas de entrada/salida
DIR_IN      = Path("../traits/output_csv")
NEXUS_PATH  = Path("../trees/BEAST_posterior.nex")
TRAITS_OUT  = Path("../traits/traits_low-level.csv")


# Bloque 2 — Leer los 5 parámetros y construir tabla de rasgos
# Archivos de entrada
param1 = DIR_IN / "A.csv"   # A
param2 = DIR_IN / "SF.csv"  # SF
param3 = DIR_IN / "H.csv"   # H
param4 = DIR_IN / "TC.csv"  # TC
param5 = DIR_IN / "SC.csv"  # SC

# Cargar y quedarme solo con species + mediana
df1 = pd.read_csv(param1)[["species", "A_hat_median"]].rename(columns={"A_hat_median": "A"})
df2 = pd.read_csv(param2)[["species", "SF_hat_median"]].rename(columns={"SF_hat_median": "SF"})
df3 = pd.read_csv(param3)[["species", "H_hat_median"]].rename(columns={"H_hat_median": "H"})
df4 = pd.read_csv(param4)[["species", "TC_hat_median"]].rename(columns={"TC_hat_median": "TC"})
df5 = pd.read_csv(param5)[["species", "SC_hat_median_hz"]].rename(columns={"SC_hat_median_hz": "SC"})

# Merge progresivo por species
df_traits = (
    df1.merge(df2, on="species")
       .merge(df3, on="species")
       .merge(df4, on="species")
       .merge(df5, on="species")
)


# Bloque 3 — Utilidades para leer TAXLABELS del NEXUS y normalizar nombres
def get_taxlabels_from_nexus(path: Path):
    """
    Extrae los nombres de TAXLABELS del archivo NEXUS
    en el orden en que aparecen.
    """
    taxa = []
    in_block = False

    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line_stripped = line.strip()

            # Inicio de TAXLABELS
            if not in_block and "TAXLABELS" in line_stripped.upper():
                in_block = True
                after = line_stripped.split("TAXLABELS", 1)[1]
                after = after.replace(";", " ")
                taxa.extend(after.split())
                continue

            # Líneas siguientes hasta el ';'
            if in_block:
                line_stripped = line_stripped.replace(";", " ")
                parts = line_stripped.split()
                taxa.extend(parts)
                if ";" in line:
                    break

    taxa = [t.strip().strip("'").strip('"') for t in taxa if t.strip()]
    return taxa


def norm_name(name: str) -> str:
    """
    Normaliza nombre de especie para hacer match:
    - '_' -> ' '
    - colapsa espacios
    - minúsculas
    """
    return " ".join(name.replace("_", " ").split()).lower()


# Bloque 4 — Reordenar según TAXLABELS y guardar traits_low-level.csv
# 4.1 Leer orden de TAXLABELS desde el NEXUS
taxlabels = get_taxlabels_from_nexus(NEXUS_PATH)
taxlabels_norm = [norm_name(t) for t in taxlabels]

# 4.2 Crear clave normalizada en df_traits y usarla como índice
df_traits["key"] = df_traits["species"].apply(norm_name)
df_traits = df_traits.set_index("key")

# 4.3 Reordenar filas según el orden de TAXLABELS (usando clave normalizada)
df_ordered = df_traits.loc[taxlabels_norm].copy()

# 4.4 Ajustar columna species para que coincida exactamente con TAXLABELS (con guion bajo)
df_ordered["species"] = taxlabels

# 4.5 Restablecer índice entero y guardar CSV final
df_ordered = df_ordered.reset_index(drop=True)
df_ordered.to_csv(TRAITS_OUT, index=False)

print("Archivo final guardado en:", TRAITS_OUT)
